In [1]:
# 1. CARGA Y LIMPIEZA INICIAL
import pandas as pd
import re
import os
from pathlib import Path

# Configuración de rutas
RAW_PATH = Path("../data/raw/Resume/Resume.csv") 
PROCESSED_PATH = Path("../data/processed/")
PROCESSED_PATH.mkdir(exist_ok=True)

# Carga de datos
print("Cargando datos...")
if RAW_PATH.exists():
    df = pd.read_csv(RAW_PATH)
    print(f"Dataset cargado: {len(df)} registros")
    print(f"Columnas disponibles: {list(df.columns)}")
else:
    print(f"ERROR: No se encuentra el archivo {RAW_PATH}")
    print("Asegúrate de tener el dataset en la carpeta correcta")

# Limpieza básica de texto
if "Resume_str" in df.columns:
    df["text"] = df["Resume_str"].fillna("").str.replace(r"\s+", " ", regex=True).str.strip()
    print("Texto limpiado correctamente")
elif "Resume" in df.columns:
    df["text"] = df["Resume"].fillna("").str.replace(r"\s+", " ", regex=True).str.strip()
    print("Usando columna 'Resume' como texto principal")
else:
    print("ERROR: No se encuentra columna de texto del CV")
    print("Columnas disponibles:", df.columns.tolist())



Cargando datos...
Dataset cargado: 2484 registros
Columnas disponibles: ['ID', 'Resume_str', 'Resume_html', 'Category']
Texto limpiado correctamente


In [2]:
df = df[
    df['Category'].str.contains('engineer', case=False, na=False)
]

In [15]:
# 2. ANONIMIZACIÓN DE DATOS SENSIBLES
try:
    from presidio_analyzer import AnalyzerEngine
    from presidio_anonymizer import AnonymizerEngine
    
    analyzer = AnalyzerEngine()
    anonymizer = AnonymizerEngine()
    
    def anonymize_text(text):
        """Anonimiza información personal en el texto del CV"""
        if not text or len(text.strip()) < 10:
            return text
        
        try:
            # Análisis de entidades sensibles
            results = analyzer.analyze(text=text, language="en")
            
            # Anonimización
            anonymized = anonymizer.anonymize(text=text, analyzer_results=results)
            return anonymized.text
        except Exception as e:
            print(f"Error en anonimización: {e}")
            return text
    
    # Aplicar anonimización a una muestra para probar
    if 'text' in df.columns and len(df) > 0:
        print("Probando anonimización en los primeros 5 registros...")
        df_sample = df.head(5).copy()
        df_sample["text_anonymized"] = df_sample["text"].apply(anonymize_text)
        print("Anonimización funcionando correctamente")
        
        # Solo proceder con dataset completo si la muestra funciona
        print("Aplicando anonimización a todo el dataset...")
        df["text_anonymized"] = df["text"].apply(anonymize_text)
        print("Anonimización completada")
    else:
        print("SALTANDO: Anonimización - no hay texto para procesar")
        df["text_anonymized"] = df.get("text", "")
        
except ImportError:
    print("WARNING: Presidio no está instalado. Saltando anonimización...")
    print("Para instalar: pip install presidio-analyzer presidio-anonymizer")
    df["text_anonymized"] = df.get("text", "")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Probando anonimización en los primeros 5 registros...
Anonimización funcionando correctamente
Aplicando anonimización a todo el dataset...


KeyboardInterrupt: 

In [3]:
# 3. EXTRACCIÓN ESTRUCTURADA CON LLM
import json
import tenacity
import os
import dotenv
dotenv.load_dotenv()

# Esquema JSON objetivo para extracción
SCHEMA_TEMPLATE = {
    "education_level": None,        # "PhD | Master | Bachelor | High School | Other"
    "discipline": None,             # Campo de estudio/especialización
    "years_total_experience": None, # Años totales de experiencia (número)
    "years_skill_main": None,       # Años en habilidad principal (número)
    "current_job_title": None,      # Título actual del trabajo
    "skills": [],                   # Lista de habilidades
    "languages": {},                # {"English": "C2", "Spanish": "B2"}
    "certifications": [],           # Lista de certificaciones
    "num_promotions": None,         # Número de promociones (número)
    "avg_tenure_months": None,      # Promedio de duración en empleos en meses (número)
    "gap_months_last5y": None,      # Meses de brecha en últimos 5 años (número)
    "location_city": None,          # Ciudad de residencia
    "work_authorization": None,     # Estado de autorización de trabajo
    "gender": None,
    "age_range": None
    
    # Nota: El campo 'apto' se agregará después de la extracción basado en reglas de negocio
}

SYSTEM_PROMPT = f"""
Eres un extractor experto de CV. Analiza el texto del currículum y devuelve ÚNICAMENTE un JSON válido con este esquema exacto:

{json.dumps(SCHEMA_TEMPLATE, indent=2)}

INSTRUCCIONES CRÍTICAS:
1. Devuelve SOLO el JSON, sin explicaciones ni comentarios
2. Si un campo no está presente o no se puede determinar, inventatelo con coherencia
3. Para arrays vacíos usa []
4. Para objects vacíos usa {{}}
5. Los números deben ser enteros, no strings
6. education_level debe ser uno de: "PhD", "Master", "Bachelor", "Associate", "High School", "Other"
7. Para languages usa códigos como "A1", "A2", "B1", "B2", "C1", "C2" o "Native"
8. NO inventes datos que no estén en el texto
9. Para engineering roles, enfócate en habilidades técnicas relevantes 
"""

try:
    from openai import AzureOpenAI
    
    # Configuración del cliente Azure OpenAI
    client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        api_key=os.environ.get("AZURE_OPENAI_KEY", ""),
        api_version="2024-02-15-preview"  # Versión estable
    )
    
    print("Cliente Azure OpenAI configurado correctamente")
    
except ImportError:
    print("WARNING: OpenAI library no está disponible")
    print("Para instalar: pip install openai")
    
    # Cliente mock para desarrollo/testing
    class MockOpenAI:
        class chat:
            class completions:
                @staticmethod
                def create(**kwargs):
                    class MockResponse:
                        class choices:
                            class message:
                                content = json.dumps(SCHEMA_TEMPLATE)
                        choices = [choices()]
                    return MockResponse()
    
    client = MockOpenAI()
    
except Exception as e:
    print(f"ERROR configurando Azure OpenAI: {e}")
    print("Verifica las variables de entorno AZURE_OPENAI_ENDPOINT y AZURE_OPENAI_KEY")
    
    # Cliente mock para desarrollo/testing
    class MockOpenAI:
        class chat:
            class completions:
                @staticmethod
                def create(**kwargs):
                    class MockResponse:
                        class choices:
                            class message:
                                content = json.dumps(SCHEMA_TEMPLATE)
                        choices = [choices()]
                    return MockResponse()
    
    client = MockOpenAI()

@tenacity.retry(
    wait=tenacity.wait_random_exponential(min=1, max=10),
    stop=tenacity.stop_after_attempt(3),
    retry=tenacity.retry_if_exception_type((Exception,))
)
def extract_features(text: str) -> str:
    """Extrae características estructuradas del texto del CV"""
    if not text or len(text.strip()) < 50:
        return json.dumps(SCHEMA_TEMPLATE)
        
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f'Analiza este CV:\n\n"""{text[:4000]}"""'}  # Limitar a 4000 chars
        ],
        model="gpt-4o-mini",  # Tu deployment name en Azure
        temperature=0,
        max_tokens=1000,
        response_format={"type": "json_object"}
    )
    
    return response.choices[0].message.content


        



Cliente Azure OpenAI configurado correctamente


In [8]:
print(df.shape)
print(df.columns)
print(df.head(2))

(118, 5)
Index(['ID', 'Resume_str', 'Resume_html', 'Category', 'text'], dtype='object')
            ID                                         Resume_str  \
1690  14206561           ENGINEERING TECHNICIAN           High...   
1691  15139979           ENGINEERING ASSISTANT       Summary  ...   

                                            Resume_html     Category  \
1690  <div class="fontsize fontface vmargins hmargin...  ENGINEERING   
1691  <div class="RNA skn-cnt4 fontsize fontface vma...  ENGINEERING   

                                                   text  
1690  ENGINEERING TECHNICIAN Highlights PC Operating...  
1691  ENGINEERING ASSISTANT Summary Knowledgeable En...  


In [9]:

# 4. PROCESAMIENTO EN PARALELO Y MANEJO DE ERRORES
import concurrent.futures as cf
from tqdm import tqdm
import time

def safe_extract_features(text):
    """Función segura para extraer características con manejo de errores"""
    try:
        result_json = extract_features(text)
        parsed_result = json.loads(result_json)
        
        # Validaciones básicas del esquema
        if not isinstance(parsed_result, dict):
            return {"error": "El resultado no es un diccionario válido"}
            
        # Asegurar que los campos numéricos sean números o null
        numeric_fields = ["years_total_experience", "years_skill_main", "num_promotions", 
                         "avg_tenure_months", "gap_months_last5y"]
        
        for field in numeric_fields:
            if field in parsed_result and parsed_result[field] is not None:
                try:
                    parsed_result[field] = int(float(parsed_result[field]))
                except (ValueError, TypeError):
                    parsed_result[field] = None
        
        return parsed_result
        
    except json.JSONDecodeError as e:
        return {"error": f"JSON inválido: {str(e)}"}
    except Exception as e:
        return {"error": f"Error en extracción: {str(e)}"}

# Procesamiento del dataset
if 'text' in df.columns and len(df) > 0:
    print(f"Iniciando extracción de características para {len(df)} CVs...")
    print("Esto puede tomar varios minutos dependiendo del tamaño del dataset y la API...")
    
    max_workers = min(5, len(df))
    with cf.ThreadPoolExecutor(max_workers=max_workers) as executor:
        features_list = list(tqdm(
            executor.map(safe_extract_features, df["text"]), 
            total=len(df),
            desc="Extrayendo características"
        ))
    
    print("Extracción completada!")
    
    # Separar resultados exitosos de errores
    successful_extractions = []
    failed_extractions = []
    
    for i, result in enumerate(features_list):
        if "error" in result:
            failed_extractions.append({"index": i, "error": result["error"]})
        else:
            successful_extractions.append(result)
    
    print(f"Extracciones exitosas: {len(successful_extractions)}")
    print(f"Extracciones fallidas: {len(failed_extractions)}")
    
    # Guardar errores para revisión
    if failed_extractions:
        errors_df = pd.DataFrame(failed_extractions)
        errors_df.to_csv(PROCESSED_PATH / "extraction_errors.csv", index=False)
        print("Errores guardados en extraction_errors.csv")
    
    # Crear DataFrame con características extraídas
    if successful_extractions:
        features_df = pd.json_normalize(successful_extractions)
        print(f"DataFrame de características creado: {features_df.shape}")
        print("Columnas extraídas:", list(features_df.columns))
        
        # AGREGAR CAMPO 'APTO' BASADO EN REGLAS DE NEGOCIO
        print("\nAplicando reglas para determinar candidatos aptos...")
        
        def evaluar_aptitud_ingeniero(row):
            """
            Evalúa si un candidato es apto para un puesto de ingeniero
            Retorna: 1 (apto), 0 (no apto), -1 (necesita revisión manual)
            """
            score = 0
            
            # 1. Experiencia mínima (peso: 3 puntos)
            years_exp = row.get('years_total_experience', 0) or 0
            if years_exp >= 5:
                score += 3
            elif years_exp >= 2:
                score += 2
            elif years_exp >= 1:
                score += 1
            
            # 2. Nivel educativo (peso: 2 puntos)
            education = row.get('education_level', '')
            if education in ['PhD', 'Master']:
                score += 2
            elif education == 'Bachelor':
                score += 1
            
            # 3. Skills técnicos relevantes (peso: 3 puntos)
            skills = row.get('skills', []) or []
            if isinstance(skills, list):
                technical_skills = [
                    'python', 'java', 'c++', 'javascript', 'sql', 'aws', 'azure',
                    'machine learning', 'data analysis', 'software development',
                    'project management', 'agile', 'scrum', 'git', 'docker'
                ]
                skills_text = ' '.join(skills).lower()
                matching_skills = sum(1 for skill in technical_skills if skill in skills_text)
                
                if matching_skills >= 3:
                    score += 3
                elif matching_skills >= 2:
                    score += 2
                elif matching_skills >= 1:
                    score += 1
            
            # 4. Certificaciones (peso: 1 punto)
            certifications = row.get('certifications', []) or []
            if isinstance(certifications, list) and len(certifications) > 0:
                score += 1
            
            # 5. Idiomas (peso: 1 punto si incluye inglés)
            languages = row.get('languages', {}) or {}
            if isinstance(languages, dict):
                english_level = languages.get('English', '').upper()
                if english_level in ['B2', 'C1', 'C2', 'NATIVE']:
                    score += 1
            
            # Criterios de decisión
            if score >= 7:
                return 1    # Apto
            elif score >= 4:
                return -1   # Necesita revisión manual
            else:
                return 0    # No apto
        
        # Aplicar evaluación
        features_df['apto'] = features_df.apply(evaluar_aptitud_ingeniero, axis=1)
        
        # Estadísticas de aptitud
        aptitud_counts = features_df['apto'].value_counts()
        total = len(features_df)
        
        print(f"Evaluación de aptitud completada:")
        print(f"  Aptos (1): {aptitud_counts.get(1, 0)} ({aptitud_counts.get(1, 0)/total*100:.1f}%)")
        print(f"  No aptos (0): {aptitud_counts.get(0, 0)} ({aptitud_counts.get(0, 0)/total*100:.1f}%)")
        print(f"  Revisión manual (-1): {aptitud_counts.get(-1, 0)} ({aptitud_counts.get(-1, 0)/total*100:.1f}%)")
        
    else:
        print("ERROR: No se pudo extraer ninguna característica")
        features_df = pd.DataFrame()
        
else:
    print("SALTANDO: Extracción - no hay texto anonimizado para procesar")
    features_df = pd.DataFrame()

# Guardar dataset con características extraídas y campo apto
if len(features_df) > 0:
    features_df.to_parquet(PROCESSED_PATH / "features_extracted.parquet", index=False)
    features_df.to_csv(PROCESSED_PATH / "features_extracted.csv", index=False)
    print(f"\nDataset con características guardado en:")
    print(f"  Parquet: {PROCESSED_PATH / 'features_extracted.parquet'}")
    print(f"  CSV: {PROCESSED_PATH / 'features_extracted.csv'}")
else:
    print("No hay datos para guardar")


Iniciando extracción de características para 118 CVs...
Esto puede tomar varios minutos dependiendo del tamaño del dataset y la API...


Extrayendo características:   0%|          | 0/118 [00:00<?, ?it/s]

Extrayendo características: 100%|██████████| 118/118 [01:23<00:00,  1.42it/s]

Extracción completada!
Extracciones exitosas: 118
Extracciones fallidas: 0
DataFrame de características creado: (118, 21)
Columnas extraídas: ['education_level', 'discipline', 'years_total_experience', 'years_skill_main', 'current_job_title', 'skills', 'certifications', 'num_promotions', 'avg_tenure_months', 'gap_months_last5y', 'location_city', 'work_authorization', 'gender', 'age_range', 'languages.English', 'languages.Spanish', 'languages.Hindi', 'languages.Armenian', 'languages.German', 'languages.Turkish', 'languages.Mandarin']

Aplicando reglas para determinar candidatos aptos...
Evaluación de aptitud completada:
  Aptos (1): 19 (16.1%)
  No aptos (0): 13 (11.0%)
  Revisión manual (-1): 86 (72.9%)

Dataset con características guardado en:
  Parquet: ..\data\processed\features_extracted.parquet
  CSV: ..\data\processed\features_extracted.csv


In [10]:
# 5. NORMALIZACIÓN, VALIDACIÓN Y ENRIQUECIMIENTO
import numpy as np

# Combinar datos originales con características extraídas
if len(features_df) > 0:
    print("Combinando datos originales con características extraídas...")
    
    # Combinar solo si tenemos el mismo número de registros exitosos
    if len(features_df) == len(df):
        enriched = pd.concat([
            df.drop(columns=["text", "text_anonymized"], errors='ignore'), 
            features_df
        ], axis=1)
    else:
        # Si hay menos características que registros originales debido a errores
        print(f"WARNING: {len(df)} registros originales vs {len(features_df)} características extraídas")
        enriched = features_df.copy()
    
    print(f"Dataset enriquecido creado: {enriched.shape}")
    
    # VALIDACIÓN Y LIMPIEZA DE DATOS
    print("Aplicando validaciones y limpieza...")
    
    # 1. Campos numéricos: limitar a rangos razonables
    numeric_validations = {
        "years_total_experience": (0, 50),    # Máximo 50 años de experiencia
        "years_skill_main": (0, 30),          # Máximo 30 años en habilidad principal  
        "num_promotions": (0, 20),            # Máximo 20 promociones
        "avg_tenure_months": (0, 600),        # Máximo 50 años (600 meses) en un trabajo
        "gap_months_last5y": (0, 60)          # Máximo 5 años de brecha
    }
    
    for col, (min_val, max_val) in numeric_validations.items():
        if col in enriched.columns:
            # Convertir a numérico y aplicar límites
            enriched[col] = pd.to_numeric(enriched[col], errors='coerce')
            enriched[col] = enriched[col].clip(min_val, max_val)
    
    # 2. Limpiar education_level: estandarizar valores
    if "education_level" in enriched.columns:
        education_mapping = {
            "phd": "PhD", "doctorate": "PhD", "doctoral": "PhD",
            "master": "Master", "masters": "Master", "mba": "Master",
            "bachelor": "Bachelor", "bachelors": "Bachelor", "ba": "Bachelor", "bs": "Bachelor",
            "associate": "Associate", "associates": "Associate",
            "high school": "High School", "highschool": "High School",
            "diploma": "High School"
        }
        
        enriched["education_level"] = enriched["education_level"].astype(str).str.lower()
        enriched["education_level"] = enriched["education_level"].replace(education_mapping)
        enriched["education_level"] = enriched["education_level"].replace("nan", None)
    
    # 3. Control de calidad: revisar nulos
    null_percentages = enriched.isnull().mean() * 100
    print("\nPorcentaje de valores nulos por columna:")
    for col, null_pct in null_percentages.items():
        if null_pct > 25:
            print(f"  {col}: {null_pct:.1f}% (ALTO)")
        elif null_pct > 0:
            print(f"  {col}: {null_pct:.1f}%")
    
    # Verificar que no hay demasiados nulos
    critical_fields = ["current_job_title", "skills"]
    for field in critical_fields:
        if field in enriched.columns:
            null_pct = enriched[field].isnull().mean() * 100
            if null_pct > 50:
                print(f"WARNING: Campo crítico '{field}' tiene {null_pct:.1f}% de nulos")
    
else:
    print("ERROR: No hay características extraídas para procesar")
    enriched = df.copy()


Combinando datos originales con características extraídas...
Dataset enriquecido creado: (236, 26)
Aplicando validaciones y limpieza...

Porcentaje de valores nulos por columna:
  ID: 50.0% (ALTO)
  Resume_str: 50.0% (ALTO)
  Resume_html: 50.0% (ALTO)
  Category: 50.0% (ALTO)
  education_level: 50.0% (ALTO)
  discipline: 50.4% (ALTO)
  years_total_experience: 50.0% (ALTO)
  years_skill_main: 50.0% (ALTO)
  current_job_title: 50.0% (ALTO)
  skills: 50.0% (ALTO)
  certifications: 50.0% (ALTO)
  num_promotions: 50.0% (ALTO)
  avg_tenure_months: 50.0% (ALTO)
  gap_months_last5y: 50.0% (ALTO)
  location_city: 50.0% (ALTO)
  work_authorization: 50.0% (ALTO)
  gender: 52.1% (ALTO)
  age_range: 50.0% (ALTO)
  languages.English: 55.5% (ALTO)
  languages.Spanish: 97.9% (ALTO)
  languages.Hindi: 98.3% (ALTO)
  languages.Armenian: 99.6% (ALTO)
  languages.German: 99.6% (ALTO)
  languages.Turkish: 99.6% (ALTO)
  languages.Mandarin: 99.6% (ALTO)
  apto: 50.0% (ALTO)


In [7]:
# 6. FILTRADO ESPECÍFICO PARA INGENIEROS
print("\n" + "="*50)
print("FILTRADO PARA PUESTOS DE INGENIERÍA")
print("="*50)

# Importaciones y configuración de rutas (en caso de que no estén disponibles)
import pandas as pd
from pathlib import Path

# Configurar rutas si no están definidas
try:
    PROCESSED_PATH
except NameError:
    PROCESSED_PATH = Path("../data/processed/")
    print(f"📁 Configurando ruta de datos procesados: {PROCESSED_PATH}")

# Verificar si tenemos datos para procesar
print("Verificando disponibilidad de datos...")

# Intentar cargar datos desde diferentes fuentes
enriched = None

# 1. Verificar si existe la variable 'enriched' del procesamiento anterior
try:
    if 'enriched' in globals():
        # Verificar que la variable existe y no es None
        existing_enriched = globals()['enriched']
        if existing_enriched is not None and len(existing_enriched) > 0:
            enriched = existing_enriched
            print(f"✅ Usando variable 'enriched' de la sesión actual: {len(enriched)} registros")
        else:
            raise NameError("Variable 'enriched' existe pero está vacía o es None")
    else:
        raise NameError("Variable 'enriched' no disponible")
except (NameError, TypeError):
    print("⚠️ Variable 'enriched' no disponible en la sesión actual")
    
    # 2. Intentar cargar desde archivo parquet procesado
    try:
        features_path = PROCESSED_PATH / "features_extracted.parquet"
        if features_path.exists():
            enriched = pd.read_parquet(features_path)
            print(f"✅ Datos cargados desde archivo parquet: {len(enriched)} registros")
        else:
            # 3. Intentar cargar desde archivo CSV como respaldo
            csv_path = PROCESSED_PATH / "features_extracted.csv"
            if csv_path.exists():
                enriched = pd.read_csv(csv_path)
                print(f"✅ Datos cargados desde archivo CSV: {len(enriched)} registros")
            else:
                raise FileNotFoundError("No se encontraron archivos de datos procesados")
    except Exception as e:
        print(f"❌ Error cargando datos desde archivos: {e}")
        print("💡 Ejecuta las celdas anteriores para generar los datos")
        enriched = pd.DataFrame()

# Continuar solo si tenemos datos
if enriched is not None and len(enriched) > 0:
    # Definir términos relacionados con ingeniería
    engineering_keywords = {
        'disciplines': [
            'engineering', 'engineer', 'mechanical', 'electrical', 'civil', 'chemical', 
            'software', 'computer science', 'information technology', 'data science',
            'systems', 'industrial', 'aerospace', 'biomedical', 'environmental',
            'structural', 'automotive', 'telecommunications', 'robotics'
        ],
        'job_titles': [
            'engineer', 'developer', 'architect', 'analyst', 'scientist', 'programmer',
            'technician', 'specialist', 'consultant', 'manager', 'lead', 'senior',
            'principal', 'staff', 'chief technology', 'technical director'
        ],
        'skills': [
            'python', 'java', 'c++', 'matlab', 'autocad', 'solidworks', 'aws', 'azure',
            'machine learning', 'artificial intelligence', 'data analysis', 'sql',
            'project management', 'agile', 'scrum', 'tensorflow', 'kubernetes'
        ]
    }

    def is_engineering_profile(row):
        """Determina si un perfil es de ingeniería basado en múltiples campos"""
        score = 0
        
        # Verificar discipline
        if pd.notna(row.get('discipline', '')):
            discipline = str(row['discipline']).lower()
            if any(keyword in discipline for keyword in engineering_keywords['disciplines']):
                score += 3
        
        # Verificar current_job_title
        if pd.notna(row.get('current_job_title', '')):
            job_title = str(row['current_job_title']).lower()
            if any(keyword in job_title for keyword in engineering_keywords['job_titles']):
                score += 2
        
        # Verificar skills (si es una lista o string)
        skills_to_check = row.get('skills', [])
        if pd.notna(skills_to_check):
            if isinstance(skills_to_check, list):
                skills_text = ' '.join(str(skill) for skill in skills_to_check).lower()
            else:
                skills_text = str(skills_to_check).lower()
                
            matching_skills = sum(1 for keyword in engineering_keywords['skills'] 
                                if keyword in skills_text)
            score += min(matching_skills, 2)  # Máximo 2 puntos por skills
        
        # Verificar education_level (preferir títulos técnicos)
        if row.get('education_level') in ['Bachelor', 'Master', 'PhD']:
            score += 1
        
        return score >= 3  # Threshold mínimo para considerar como ingeniero

    # Aplicar filtrado
    print(f"Dataset antes del filtrado: {len(enriched)} registros")
    
    # Aplicar función de filtrado
    print("Aplicando filtros de ingeniería...")
    engineering_mask = enriched.apply(is_engineering_profile, axis=1)
    engineers_df = enriched[engineering_mask].copy()
    
    print(f"Perfiles de ingeniería identificados: {len(engineers_df)} registros")
    print(f"Porcentaje de ingenieros: {len(engineers_df)/len(enriched)*100:.1f}%")
    
    # Mostrar estadísticas de los perfiles filtrados
    if len(engineers_df) > 0:
        print("\nESTADÍSTICAS DE PERFILES DE INGENIERÍA:")
        
        # Education levels
        if 'education_level' in engineers_df.columns:
            education_counts = engineers_df['education_level'].value_counts(dropna=False)
            print(f"\nNiveles de educación:")
            for edu, count in education_counts.items():
                print(f"  {edu}: {count} ({count/len(engineers_df)*100:.1f}%)")
        
        # Disciplinas más comunes
        if 'discipline' in engineers_df.columns:
            discipline_counts = engineers_df['discipline'].value_counts(dropna=False).head(10)
            print(f"\nTop 10 disciplinas:")
            for disc, count in discipline_counts.items():
                print(f"  {disc}: {count}")
        
        # Experiencia promedio
        if 'years_total_experience' in engineers_df.columns:
            avg_exp = engineers_df['years_total_experience'].mean()
            if pd.notna(avg_exp):
                print(f"\nExperiencia promedio: {avg_exp:.1f} años")
        
        # Estadísticas del campo 'apto' si existe
        if 'apto' in engineers_df.columns:
            aptitud_counts = engineers_df['apto'].value_counts()
            total = len(engineers_df)
            
            print(f"\nESTADÍSTICAS DE APTITUD:")
            labels = {1: 'Aptos', 0: 'No aptos', -1: 'Revisión manual'}
            for value, count in aptitud_counts.items():
                label = labels.get(value, f'Valor {value}')
                percentage = count / total * 100
                print(f"  {label}: {count} candidatos ({percentage:.1f}%)")
    
    # Dataset final para entrenamiento
    final_dataset = engineers_df.copy()
    
    # Guardar dataset filtrado
    if len(final_dataset) > 0:
        print(f"\n✅ FILTRADO COMPLETADO EXITOSAMENTE")
        print(f"📊 Dataset de ingenieros listo: {len(final_dataset)} registros")
    
else:
    print("❌ ERROR: No hay datos enriquecidos para filtrar")
    print("💡 Asegúrate de ejecutar las celdas anteriores primero")
    final_dataset = pd.DataFrame()



FILTRADO PARA PUESTOS DE INGENIERÍA
📁 Configurando ruta de datos procesados: ..\data\processed
Verificando disponibilidad de datos...
⚠️ Variable 'enriched' no disponible en la sesión actual
✅ Datos cargados desde archivo parquet: 118 registros
Dataset antes del filtrado: 118 registros
Aplicando filtros de ingeniería...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
# 7. EXPORTACIÓN Y VISUALIZACIÓN FINAL
print("\n" + "="*50)
print("EXPORTACIÓN Y ANÁLISIS FINAL")
print("="*50)

# Verificar si tenemos el dataset final
try:
    if 'final_dataset' in globals() and len(final_dataset) > 0:
        print(f"✅ Dataset final disponible: {len(final_dataset)} registros")
    else:
        raise NameError("Variable 'final_dataset' no disponible")
except NameError:
    print("⚠️ Variable 'final_dataset' no disponible")
    print("💡 Intentando cargar desde archivos existentes...")
    
    # Intentar cargar dataset de ingenieros si existe
    try:
        engineers_path = PROCESSED_PATH / "engineers_dataset_for_training.parquet"
        if engineers_path.exists():
            final_dataset = pd.read_parquet(engineers_path)
            print(f"✅ Dataset cargado desde archivo: {len(final_dataset)} registros")
        else:
            # Usar el dataset general de características
            features_path = PROCESSED_PATH / "features_extracted.parquet"
            if features_path.exists():
                final_dataset = pd.read_parquet(features_path)
                print(f"✅ Usando dataset de características: {len(final_dataset)} registros")
            else:
                final_dataset = pd.DataFrame()
                print("❌ No se encontraron archivos de datos")
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        final_dataset = pd.DataFrame()

if len(final_dataset) > 0:
    # Guardar dataset final
    final_parquet_path = PROCESSED_PATH / "engineers_dataset_for_training.parquet"
    final_csv_path = PROCESSED_PATH / "engineers_dataset_for_training.csv"
    
    final_dataset.to_parquet(final_parquet_path, index=False)
    final_dataset.to_csv(final_csv_path, index=False)
    
    print(f"Dataset final guardado:")
    print(f"  Parquet: {final_parquet_path}")
    print(f"  CSV: {final_csv_path}")
    print(f"  Registros: {len(final_dataset)}")
    print(f"  Columnas: {len(final_dataset.columns)}")
    
    # Resumen del esquema final
    print(f"\nESQUEMA DEL DATASET FINAL:")
    for col in final_dataset.columns:
        dtype = final_dataset[col].dtype
        null_count = final_dataset[col].isnull().sum()
        null_pct = null_count / len(final_dataset) * 100
        print(f"  {col:25s} | {str(dtype):15s} | {null_count:4d} nulos ({null_pct:5.1f}%)")
    
    # Estadísticas del campo 'apto'
    if 'apto' in final_dataset.columns:
        print(f"\nESTADÍSTICAS DEL CAMPO 'APTO':")
        aptitud_counts = final_dataset['apto'].value_counts()
        total = len(final_dataset)
        
        labels = {1: 'Aptos', 0: 'No aptos', -1: 'Revisión manual'}
        for value, count in aptitud_counts.items():
            label = labels.get(value, f'Valor {value}')
            percentage = count / total * 100
            print(f"  {label}: {count} candidatos ({percentage:.1f}%)")
    
    # Muestra de datos
    print(f"\nMUESTRA DE LOS PRIMEROS 3 REGISTROS:")
    sample_cols = ['current_job_title', 'discipline', 'years_total_experience', 'education_level', 'apto']
    available_cols = [col for col in sample_cols if col in final_dataset.columns]
    if available_cols:
        print(final_dataset[available_cols].head(3).to_string(index=False))
    
    # Generar histogramas y visualizaciones si hay matplotlib
    try:
        import matplotlib.pyplot as plt
        
        # Visualizaciones numéricas
        numeric_cols = ['years_total_experience', 'years_skill_main', 'num_promotions', 
                       'avg_tenure_months', 'gap_months_last5y']
        available_numeric = [col for col in numeric_cols if col in final_dataset.columns]
        
        if available_numeric:
            fig, axes = plt.subplots(2, 3, figsize=(15, 10))
            axes = axes.flatten()
            
            for i, col in enumerate(available_numeric[:5]):
                data = final_dataset[col].dropna()
                if len(data) > 0:
                    axes[i].hist(data, bins=min(20, len(data.unique())), alpha=0.7, edgecolor='black')
                    axes[i].set_title(f'{col}\n(n={len(data)}, avg={data.mean():.1f})')
                    axes[i].set_xlabel(col)
                    axes[i].set_ylabel('Frecuencia')
            
            # Gráfico de barras para el campo 'apto' en el último subplot
            if 'apto' in final_dataset.columns:
                aptitud_counts = final_dataset['apto'].value_counts().sort_index()
                labels = {1: 'Aptos', 0: 'No aptos', -1: 'Revisión\nmanual'}
                
                x_labels = [labels.get(val, str(val)) for val in aptitud_counts.index]
                colors = ['red', 'orange', 'green']
                
                axes[5].bar(x_labels, aptitud_counts.values, 
                           color=colors[:len(aptitud_counts)], alpha=0.7, edgecolor='black')
                axes[5].set_title(f'Distribución Aptitud\n(Total: {len(final_dataset)})')
                axes[5].set_ylabel('Frecuencia')
                
                # Añadir valores en las barras
                for i, v in enumerate(aptitud_counts.values):
                    axes[5].text(i, v + 0.5, str(v), ha='center', va='bottom')
            else:
                axes[5].set_visible(False)
            
            # Ocultar subplot vacío si es necesario
            if len(available_numeric) < 5:
                for i in range(len(available_numeric), 5):
                    axes[i].set_visible(False)
            
            plt.tight_layout()
            plt.savefig(PROCESSED_PATH / "dataset_distributions.png", dpi=150, bbox_inches='tight')
            plt.show()
            print(f"\nHistogramas guardados en: {PROCESSED_PATH / 'dataset_distributions.png'}")
        
    except ImportError:
        print("\nWARNING: matplotlib no está disponible para generar histogramas")
        print("Para instalar: pip install matplotlib")
    
    print(f"\n✅ DATASET PARA ENTRENAMIENTO DE MODELO DE SELECCIÓN LISTO!")
    print(f"📊 {len(final_dataset)} perfiles de ingeniería estructurados")
    print(f"🎯 Listo para entrenar modelo supervisado de selección de candidatos")
    
    if 'apto' in final_dataset.columns:
        aptos = (final_dataset['apto'] == 1).sum()
        no_aptos = (final_dataset['apto'] == 0).sum()
        revision = (final_dataset['apto'] == -1).sum()
        print(f"🔍 Etiquetas para entrenamiento: {aptos} aptos, {no_aptos} no aptos, {revision} para revisión")
        print(f"📈 Modelo puede entrenarse con clasificación binaria o multiclase")
    
else:
    print("❌ ERROR: No se pudo crear el dataset final")
    print("Revisa los pasos anteriores para identificar problemas")



EXPORTACIÓN Y ANÁLISIS FINAL
⚠️ Variable 'final_dataset' no disponible
💡 Intentando cargar desde archivos existentes...
✅ Usando dataset de características: 118 registros
Dataset final guardado:
  Parquet: ..\data\processed\engineers_dataset_for_training.parquet
  CSV: ..\data\processed\engineers_dataset_for_training.csv
  Registros: 118
  Columnas: 22

ESQUEMA DEL DATASET FINAL:
  education_level           | object          |    0 nulos (  0.0%)
  discipline                | object          |    1 nulos (  0.8%)
  years_total_experience    | int64           |    0 nulos (  0.0%)
  years_skill_main          | int64           |    0 nulos (  0.0%)
  current_job_title         | object          |    0 nulos (  0.0%)
  skills                    | object          |    0 nulos (  0.0%)
  certifications            | object          |    0 nulos (  0.0%)
  num_promotions            | int64           |    0 nulos (  0.0%)
  avg_tenure_months         | int64           |    0 nulos (  0.0%)
  ga